In [1]:

if True:	#imports
	import os, sys, signal
	os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
	import tensorflow as tf
	import numpy as np
	from loader import *
	from networks import *
	#from adversarial import *

def sighandler(sig, frame):
	print("\b\bInterrupted", file=sys.stderr)
	exit(-1)
signal.signal(signal.SIGINT, sighandler)

GPU = 0
DATA_SET = ""

#------------------------------------------------------------------------------------------------

def train_classifier(classifier, dataSet):
	i = 0
	best = test_res = classifier.test(dataSet, BATCH_SIZE)
	print(test_res)
	while i<1:
		i += 1
		train_res = classifier.epoch(dataSet, BATCH_SIZE)
		test_res = classifier.test(dataSet, BATCH_SIZE)
		print("EPOCH: ", i, ", TRAIN RES: ", train_res, ", TEST RES: ", test_res, sep='')
		sys.stdout.flush()
		if test_res > best:
			classifier.save("./model_data/{}_{:.3f}.ckpt".format(DATA_SET, test_res))
			best = test_res
	return classifier	

def load_classifier(classifier):
	if DATA_SET == "CIFAR" or DATA_SET == "CIFAR10":
		classifier.restore("./model_data/CIFAR10_0.800.ckpt")
	elif DATA_SET == "MNIST":
		classifier.restore("./model_data/MNIST_0.994.ckpt")

In [2]:
if True:	#imports
	import os, sys, signal
	os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
	import tensorflow as tf
	import numpy as np
	from tqdm import tqdm, trange


class AttackData(object):
	def __init__(self, img_shp, data_length):
		self.images
		self.true_lbls
		self.prev_res
		self.masks
		self.target
		self.result
		

def legalize(img, mask):
	new_img = img+mask
	new_img[new_img <-1] =-1
	new_img[new_img > 1] = 1
	legal_mask = new_img - img
	return legal_mask

class Adversarial(object):
	def __init__(self, sess, classifier, inp_shp, n_cat, epsilon, alpha):
		self.sess = sess
		self.epsilon = epsilon

		self.img = tf.placeholder(tf.float32, inp_shp)
		self.trgt = tf.placeholder(tf.float32, [n_cat])

		self._var = tf.Variable(tf.zeros(inp_shp, tf.float32))
		img = tf.expand_dims(self.img + self._var, 0)
		self.logits = classifier.model(img, y=None, trainmode=False)

		loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.trgt))
		oob  = 100*tf.reduce_mean(tf.cast(img>1, tf.float32)) + tf.reduce_mean(tf.cast(img<-1, tf.float32))
		dist = 100*tf.reduce_mean(tf.pow(self._var, 7))
		self.fgsm_gl = tf.train.GradientDescentOptimizer(alpha).minimize(loss, var_list=[self._var])
		self.iter_gl = tf.train.GradientDescentOptimizer(alpha).minimize(loss+oob+dist, var_list=[self._var])

		self.mask = epsilon*tf.sign(self._var)

		self.reset_mask = self._var.assign(np.zeros(shape=inp_shp))


	def fgsm_attack(self, img, trgt):
		gl = self.sess.run(self.fgsm_gl, feed_dict={self.img: img, self.trgt: trgt})
		mask = self.sess.run(self.mask)
		grad = self.sess.run(self._var)
		self.sess.run(self.reset_mask)
		return mask, grad

	def iter_attack(self, img, trgt, n_iter=100):
		for _ in trange(n_iter):
			gl = self.sess.run(self.iter_gl, feed_dict={self.img: img, self.trgt: trgt})
		mask = self.sess.run(self.mask)
		grad = self.sess.run(self._var)
		self.sess.run(self.reset_mask)
		return mask, grad

In [3]:
GPU = 1
DATA_SET = "MNIST"

if DATA_SET == "CIFAR" or DATA_SET == "CIFAR10":
    architecture = CIFAR_ARCHITECTURE
    dataSet = CifarLoader("../cifar-10-batches-py/")
    inp_shp = CIFAR_INP_SHP
elif DATA_SET == "MNIST":
    architecture = MNIST_ARCHITECTURE
    dataSet = MnistLoader("../MNIST_data/")
    inp_shp = MNIST_INP_SHP

config = tf.ConfigProto( device_count = {'GPU': GPU} )
sess = tf.InteractiveSession(config=config)

classifier = Classifier(sess, inp_shp, N_CAT, architecture, DEFARGS)

epsilon = 0.1
adv = Adversarial(sess, classifier, inp_shp, N_CAT, epsilon, 1)

init = tf.global_variables_initializer()
sess.run(init)
sess.graph.finalize()

train_classifier(classifier, dataSet)
#load_classifier(classifier)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
>>>  [28, 28, 1]
>>>  [28, 28, 32]
>>>  [28, 28, 64]
>>>  [14, 14, 64]
>>>  [14, 14, 128]
>>>  [7, 7, 128]
>>>  [4, 4, 128]
>>>  [2048]
>>>  [1024]
>>>  [10]


  0%|          | 0/429 [00:00<?, ?it/s]

0.0957532


100%|██████████| 429/429 [00:26<00:00, 16.57it/s]


EPOCH: 1, TRAIN RES: 0.936735, TEST RES: 0.986278


In [4]:
test_res = classifier.test(dataSet, 1)
print("model ready, with score:", test_res)

model ready, with score: 0.9863


In [15]:
i = 0
img = dataSet.test.images[i]
imgs = dataSet.test.images[i:i+1]
lbl = dataSet.test.labels[i]
trgt = np.zeros(10)
trgt[4] = 1
trgt += -lbl
np.argmax(classifier.predict(dataSet.test.images[i:i+1])[0])

7

In [16]:
mask, grad = adv.iter_attack(img, trgt)

100%|██████████| 100/100 [00:00<00:00, 368.69it/s]


In [17]:
legal_grad = legalize(img, 0.3*grad)
legal_grad = np.expand_dims(legal_grad, 0)
legal_mask = legalize(img, 3*mask)
legal_mask = np.expand_dims(legal_mask, 0)

In [18]:
dataSet.display(imgs+legal_mask)

In [19]:
dataSet.display(imgs+legal_grad)

In [20]:
classifier.predict(imgs+legal_mask)

array([[  4.90152743e-05,   6.30188093e-04,   8.52388388e-04,
          5.33843832e-03,   6.87690699e-05,   3.37692938e-04,
          3.53958342e-07,   9.90704477e-01,   5.72171528e-04,
          1.44658098e-03]], dtype=float32)

In [21]:
classifier.predict(imgs+legal_grad)

array([[ 0.05305893,  0.05447371,  0.01596328,  0.27270785,  0.01420301,
         0.42988005,  0.02299208,  0.01775072,  0.03329035,  0.08568003]], dtype=float32)

In [12]:
classifier.predict(dataSet.test.images[i:i+1])[0]

array([  4.55525964e-08,   4.70164832e-06,   5.38635459e-06,
         5.66888266e-05,   2.59520789e-07,   6.32431863e-07,
         2.74463282e-11,   9.99910116e-01,   1.84106023e-06,
         2.03543696e-05], dtype=float32)

In [13]:
legal_grad

array([[[[  0.00000000e+00],
         [  5.42479753e-03],
         [  1.29872561e-03],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  3.30746174e-04],
         [  2.94643641e-03],
         [  5.36924601e-03],
         [  4.25100327e-03],
         [  3.97443771e-04],
         [  0.00000000e+00],
         [  0.00000000e+00],
         [  0.00000000e+00]],

        [[  0.00000000e+00],
         [  1.98191404e-03],
         [  4.05836105e-03],
         [  5.10787964e-03],
         [  6.97487593e-03],
         [  6.87342882e-03],
         [  